In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./data5.csv', encoding = 'gbk')

In [3]:
a = pd.read_csv('balance.csv')
a.head(2)

,ts_code,ann_date,end_date,report_type,undistr_porfit,surplus_rese,total_cur_assets,total_assets,total_cur_liab,total_liab,total_hldr_eqy_inc_min_int
0,000006.SZ,20190330,20181231,1,2.879490e+09,1.483863e+09,1.042744e+10,1.353673e+10,5.436699e+09,7.150591e+09,6.386137e+09
1,000006.SZ,20181027,20180930,1,2.629079e+09,1.363271e+09,9.639073e+09,1.246334e+10,4.815261e+09,6.454994e+09,6.008350e+09


In [4]:
a = a[['ts_code', 'end_date', 'undistr_porfit','surplus_rese', 'total_cur_assets',
       'total_assets', 'total_cur_liab','total_liab', 'total_hldr_eqy_inc_min_int']]
a.columns = ['证券代码', '日期', '未分配利润','盈余公积','流动资产合计','资产总计',
             '流动负债合计', '负债合计','所有者权益合计']
a.head(2)

,证券代码,日期,未分配利润,盈余公积,流动资产合计,资产总计,流动负债合计,负债合计,所有者权益合计
0,000006.SZ,20181231,2.879490e+09,1.483863e+09,1.042744e+10,1.353673e+10,5.436699e+09,7.150591e+09,6.386137e+09
1,000006.SZ,20180930,2.629079e+09,1.363271e+09,9.639073e+09,1.246334e+10,4.815261e+09,6.454994e+09,6.008350e+09


In [5]:
b = pd.read_csv('income.csv')
b.head(2)

,ts_code,ann_date,end_date,report_type,revenue,fin_exp,total_profit
0,000006.SZ,20190330,20181231,1,2.511844e+09,88912716.85,1.068987e+09
1,000006.SZ,20181027,20180930,1,1.593892e+09,63814847.96,6.399546e+08


In [6]:
b = b[['ts_code', 'end_date', 'revenue', 'fin_exp','total_profit']]
b.columns = ['证券代码', '日期', '营业收入', '利润总额', '财务费用']
b.head(2)

,证券代码,日期,营业收入,利润总额,财务费用
0,000006.SZ,20181231,2.511844e+09,88912716.85,1.068987e+09
1,000006.SZ,20180930,1.593892e+09,63814847.96,6.399546e+08


In [7]:
c = pd.read_csv('stock.csv')
c.head(2)

,trade_date,ts_code,total_share,float_share,total_mv,circ_mv,index,non_float_share
0,20010928,000006.SZ,253591631.0,157807037.0,3.164824e+09,1.969432e+09,200109,95784594.0
1,20011031,000006.SZ,253591631.0,157807037.0,2.878265e+09,1.791110e+09,200110,95784594.0


In [8]:
c = c[['ts_code', 'trade_date', 'total_share','float_share', 'non_float_share','circ_mv']]
c.columns = ['证券代码', '交易日期', '总股本', '流通股本', '非流通股本','流通市值']
c.head(2)

,证券代码,交易日期,总股本,流通股本,非流通股本,流通市值
0,000006.SZ,20010928,253591631.0,157807037.0,95784594.0,1.969432e+09
1,000006.SZ,20011031,253591631.0,157807037.0,95784594.0,1.791110e+09


In [9]:
result = pd.DataFrame(df['证券代码'])
result['类型'] = df['公司属性\r\n[交易日期] 2019-09-30']

In [10]:
result = pd.merge(result,a,on=['证券代码'],how='left',copy=False)
result = pd.merge(result,b,on=['证券代码','日期'],how='left',copy=False)

In [11]:
c['交易日期'] = c['交易日期'].astype(str)
c['年份'] = c['交易日期'].apply(lambda x: x[:4])
c['月份'] = c['交易日期'].apply(lambda x: x[4:6])
result['日期'] = result['日期'].astype(str)
result['年份'] = result['日期'].apply(lambda x: x[:4])
result['月份'] = result['日期'].apply(lambda x: x[4:6])

In [12]:
result = pd.merge(result,c[['证券代码','总股本', '流通股本', '非流通股本', '流通市值','年份', '月份']],
                  on=['证券代码', '年份', '月份'],how='left',copy=False)

In [13]:
result.columns

Index(['证券代码', '类型', '日期', '未分配利润', '盈余公积', '流动资产合计', '资产总计', '流动负债合计', '负债合计',
       '所有者权益合计', '营业收入', '利润总额', '财务费用', '年份', '月份', '总股本', '流通股本', '非流通股本',
       '流通市值'],
      dtype='object')

In [14]:
result['X1'] = (result['流动资产合计'] - result['流动负债合计']) / result['资产总计']
result['X2'] = (result['盈余公积'] + result['未分配利润']) / result['资产总计']
result['X3'] = (result['利润总额'] + result['财务费用']) / result['资产总计']
result['X4'] = (result['流通市值'] + (result['所有者权益合计']/result['总股本']) * result['非流通股本']) / result['负债合计']
result['X5'] = result['营业收入']/result['资产总计']

In [15]:
result['Z'] = 1.2*result['X1'] + 1.4*result['X2']+ 3.3* result['X3'] + 0.6*result['X4'] + 1.0*result['X5']

In [18]:
print(result.duplicated().sum())
print(result.shape)
result = result.drop_duplicates()
print(result.shape)

1564
(47858, 25)
(46294, 25)


In [19]:
result[['X1','X2','X3','X4','X5','Z']].describe()

,X1,X2,X3,X4,X5,Z
count,45869.000000,4.565200e+04,4.582400e+04,42853.000000,4.627000e+04,42048.000000
mean,0.106526,-inf,inf,3.844007,inf,3.095639
std,0.278227,NaN,NaN,14.023123,NaN,9.148174
min,-8.873287,-inf,-3.587045e+00,-616.589312,0.000000e+00,-368.449721
25%,-0.042782,4.504101e-02,1.011788e-02,0.863017,1.578999e-01,1.131488
50%,0.109251,1.023258e-01,2.597531e-02,1.721416,3.213484e-01,1.944207
75%,0.272119,1.798485e-01,5.098972e-02,3.676218,5.854939e-01,3.336046
max,0.969807,8.110288e-01,inf,1735.827155,inf,1042.993297


In [20]:
result.to_csv('result_new.csv', index = False)

In [21]:
result.to_excel('result_new_excel.xlsx', index = False)

In [22]:
result.head()

,证券代码,类型,日期,未分配利润,盈余公积,流动资产合计,资产总计,流动负债合计,负债合计,所有者权益合计,...,总股本,流通股本,非流通股本,流通市值,X1,X2,X3,X4,X5,Z
0,000006.SZ,地方国有企业,20181231,2.879490e+09,1.483863e+09,1.042744e+10,1.353673e+10,5.436699e+09,7.150591e+09,6.386137e+09,...,1.349995e+09,1.348308e+09,1687074.0,6.984235e+09,0.368681,0.322334,0.085538,0.977852,0.185558,1.948228
1,000006.SZ,地方国有企业,20180930,2.629079e+09,1.363271e+09,9.639073e+09,1.246334e+10,4.815261e+09,6.454994e+09,6.008350e+09,...,1.349995e+09,1.348308e+09,1687074.0,7.685355e+09,0.387040,0.320327,0.056467,1.191769,0.127886,1.942196
2,000006.SZ,地方国有企业,20180630,2.541807e+09,1.363271e+09,9.288721e+09,1.209422e+10,4.481685e+09,6.181762e+09,5.912462e+09,...,1.349995e+09,1.348267e+09,1727639.0,7.900847e+09,0.397465,0.322888,0.045685,1.279314,0.087280,1.934632
3,000006.SZ,地方国有企业,20180331,2.656395e+09,1.363271e+09,8.340574e+09,1.113275e+10,3.921206e+09,5.138921e+09,5.993828e+09,...,1.349995e+09,1.348267e+09,1727639.0,9.936731e+09,0.396970,0.361067,0.028922,1.935115,0.043570,2.281940
4,000006.SZ,地方国有企业,20171231,2.370188e+09,1.363271e+09,1.058326e+10,1.313993e+10,6.355662e+09,7.432572e+09,5.707359e+09,...,NaN,NaN,NaN,NaN,0.321737,0.284131,0.081667,NaN,0.225226,NaN
